#### Splitting dataset

In [2]:
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
from rdkit.Chem import rdMolDescriptors as rdescriptors
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys    
import pubchempy as pcp
import warnings
warnings.filterwarnings('ignore')
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')
from sklearn.utils import shuffle
import sys
sys.path.insert(0,"./../fip3")
from fip.profiles import CooccurrenceProfile
import fip.chem
from fip import profiles
import re
from itertools import chain
from ast import literal_eval

In [2]:
#Loading zinc database from csv 
suppl_csv = pd.read_csv('../data/CP1/ZINC/ZINCrandom.csv', delimiter = ' ')

In [3]:
suppl_csv

,Zinc_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset
0,8432508,Cc1ccc(-n2c(=O)c3c(n4c(SCCc5ccccc5)nnc24)-c2cc...,532.229683,6.94842,DPUIXUVMXJXCJR-UHFFFAOYSA-N,DPUIXUVMXJXCJR,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[2, 58, 74, 80, 105, 116, 153, 233, 235, 289, ..."
1,2325339,Cc1ccc(-n2c(COc3ccccc3)nnc2SCC(=O)Nc2ccc3c(c2)...,488.151826,4.65672,HVIYVAXZIXJRQC-UHFFFAOYSA-N,HVIYVAXZIXJRQC,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[41, 58, 80, 102, 116, 127, 184, 191, 233, 235..."
2,408636482,COCCNS(=O)(=O)c1ccc(OCC(=O)N2CCC(C(N)=O)CC2)c(...,433.107434,0.36740,RAILYSNQBZZKMQ-UHFFFAOYSA-N,RAILYSNQBZZKMQ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[13, 80, 94, 111, 140, 198, 222, 279, 317, 319..."
3,97995285,Cc1ccc(S(=O)(=O)n2nc(NCc3cccc(Br)c3)nc2NCc2ccc...,588.978270,5.57272,FFUQFTXMHIMSHO-UHFFFAOYSA-N,FFUQFTXMHIMSHO,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 82, 92, 157, 235, 255, 319, 350, 374, 378..."
4,408561088,CCn1c(-c2ccccc2)c(C)s/c1=N\C(=O)c1ccccc1Br,400.024496,5.04852,GMWZBJMZXGQSQF-VZCXRCSSSA-N,GMWZBJMZXGQSQF,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[30, 34, 57, 80, 162, 289, 294, 308, 313, 351,..."
...,...,...,...,...,...,...,...,...,...
379007,39850378,CCOC(=O)C1=C(C)N=c2s/c(=C\c3ccc(OCC)c(OC)c3)c(...,478.156243,3.20560,NKVUPLXRBGDCKZ-FQWPPLDYSA-N,NKVUPLXRBGDCKZ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[25, 80, 94, 125, 145, 147, 162, 237, 249, 281..."
379008,606422253,CCCCOc1ccc(-c2nn(-c3ccccc3)cc2CNC(=O)c2ccc(OCC...,501.242770,6.57610,GRVSBYRYFLOHAZ-UHFFFAOYSA-N,GRVSBYRYFLOHAZ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[13, 76, 80, 94, 102, 197, 255, 265, 294, 322,..."
379009,19234980,CNc1cc([C@H]2CCCN(Cc3cccc(C)c3)C2)nc(-c2cccnc2)n1,373.226646,4.26832,WGJHTUZEMVLXOF-FQEVSTJZSA-N,WGJHTUZEMVLXOF,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 105, 257, 378, 437, 446, 736, 739, 744, 7..."
379010,12371082,COc1ccc(C23CC4CC(CC(C4)C2)C3)cc1C(=O)NC(c1cccc...,451.251129,6.68240,GUGKIBFTNCSUKM-UHFFFAOYSA-N,GUGKIBFTNCSUKM,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 46, 94, 117, 235, 249, 389, 484, 486, 556,..."


In [4]:
%%time
instance = []
for x in suppl_csv.Smiles:
    m = Chem.MolFromSmiles(x)
    z =fip.chem.rdmol2morgan_feature_smiles(m, radius=2, min_radius=2, all_bonds_explicit=False, canonical_smiles=True, isomeric_smiles=False, all_H_explicit=True)
    z = str(z)
    z = z.replace("{","")
    z = z.replace("}","")
    instance.append(z)

CPU times: user 1h 3min 38s, sys: 4.21 s, total: 1h 3min 42s
Wall time: 1h 3min 43s


In [6]:
suppl_csv['Fragments'] = instance

In [7]:
suppl_csv.to_csv('../data/CPF/ZINC/ZINCrandomF.csv', sep=' ', index = False)

In [8]:
suppl_csv = pd.read_csv('../data/CPF/ZINC/ZINCrandomF.csv', delimiter = ' ')

In [9]:
suppl_csv

,Zinc_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset,Fragments
0,8432508,Cc1ccc(-n2c(=O)c3c(n4c(SCCc5ccccc5)nnc24)-c2cc...,532.229683,6.94842,DPUIXUVMXJXCJR-UHFFFAOYSA-N,DPUIXUVMXJXCJR,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[2, 58, 74, 80, 105, 116, 153, 233, 235, 289, ...","'[cH][cH][c]([CH3])[cH][cH]', '[c]-[n]([c])[c]..."
1,2325339,Cc1ccc(-n2c(COc3ccccc3)nnc2SCC(=O)Nc2ccc3c(c2)...,488.151826,4.65672,HVIYVAXZIXJRQC-UHFFFAOYSA-N,HVIYVAXZIXJRQC,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[41, 58, 80, 102, 116, 127, 184, 191, 233, 235...","'[cH][cH][c]([CH3])[cH][cH]', '[c][n][n][c]([C..."
2,408636482,COCCNS(=O)(=O)c1ccc(OCC(=O)N2CCC(C(N)=O)CC2)c(...,433.107434,0.36740,RAILYSNQBZZKMQ-UHFFFAOYSA-N,RAILYSNQBZZKMQ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[13, 80, 94, 111, 140, 198, 222, 279, 317, 319...","'[O][CH2][CH2][NH][S]', '[C][CH]([CH2])[CH2][C..."
3,97995285,Cc1ccc(S(=O)(=O)n2nc(NCc3cccc(Br)c3)nc2NCc2ccc...,588.978270,5.57272,FFUQFTXMHIMSHO-UHFFFAOYSA-N,FFUQFTXMHIMSHO,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 82, 92, 157, 235, 255, 319, 350, 374, 378...","'[cH][c]([CH2])[cH][c]([cH])[Br]', '[cH][cH][c..."
4,408561088,CCn1c(-c2ccccc2)c(C)s/c1=N\C(=O)c1ccccc1Br,400.024496,5.04852,GMWZBJMZXGQSQF-VZCXRCSSSA-N,GMWZBJMZXGQSQF,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[30, 34, 57, 80, 162, 289, 294, 308, 313, 351,...",'[cH][c]([cH])-[c]([c]([CH3])[s])[n]([c])[CH2]...
...,...,...,...,...,...,...,...,...,...,...
379007,39850378,CCOC(=O)C1=C(C)N=c2s/c(=C\c3ccc(OCC)c(OC)c3)c(...,478.156243,3.20560,NKVUPLXRBGDCKZ-FQWPPLDYSA-N,NKVUPLXRBGDCKZ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[25, 80, 94, 125, 145, 147, 162, 237, 249, 281...","'[c][c]([cH])[O][CH3]', '[CH][c]([cH])[cH][c](..."
379008,606422253,CCCCOc1ccc(-c2nn(-c3ccccc3)cc2CNC(=O)c2ccc(OCC...,501.242770,6.57610,GRVSBYRYFLOHAZ-UHFFFAOYSA-N,GRVSBYRYFLOHAZ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[13, 76, 80, 94, 102, 197, 255, 265, 294, 322,...","'[C][NH][CH2][c]([c])[cH]', '[CH3][CH2][CH2][C..."
379009,19234980,CNc1cc([C@H]2CCCN(Cc3cccc(C)c3)C2)nc(-c2cccnc2)n1,373.226646,4.26832,WGJHTUZEMVLXOF-FQEVSTJZSA-N,WGJHTUZEMVLXOF,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 105, 257, 378, 437, 446, 736, 739, 744, 7...","'[c][cH][c]([n][c])[CH]([CH2])[CH2]', '[CH][CH..."
379010,12371082,COc1ccc(C23CC4CC(CC(C4)C2)C3)cc1C(=O)NC(c1cccc...,451.251129,6.68240,GUGKIBFTNCSUKM-UHFFFAOYSA-N,GUGKIBFTNCSUKM,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 46, 94, 117, 235, 249, 389, 484, 486, 556,...","'[c][CH]([NH])[c]([cH][cH])[cH][cH]', '[c][C](..."


In [17]:
suppl_csv['Fragments'][0]

"'[cH][cH][c]([CH3])[cH][cH]', '[c]-[n]([c])[c](=[O])[c]([c])[C]', '[cH][cH][c](-[c]([c])[n])[c]([cH])[CH2]', '[c][n]([c])[c]([n][n])[S][CH2]', '[cH][cH][c]([cH][cH])-[n]([c])[c]', '[cH][cH][cH][c]([c])[CH2]', '[c][C]([CH2])([CH2])[CH2][c]([c])[cH]', '[c][cH][cH][c]([cH])-[n]', '[c][C]([CH2])([CH2])[CH2][CH2][CH2]', '[cH][c]([cH])-[n]([c]([c])=[O])[c]([n])[n]', '[c]-[c]([c])[n]([c]([n])[S])[c]([n])[n]', '[cH][c]([c])-[c]([c]([c])[C])[n]([c])[c]', '[c]-[c]([c])[cH][cH][cH]', '[c]-[c]([cH])[c]([cH][cH])[CH2][C]', '[c]-[n]([c])[c]([n][n])[n]([c])[c]', '[CH2][CH2][CH2][CH2][CH2]', '[cH][cH][cH][c]([cH])[CH2]', '[c][CH2][CH2][S][c]', '[c][cH][cH][c]([cH])[CH3]', '[c][n][n][c]([n])[S]', '[cH][cH][cH][cH][cH]', '[cH][c]([cH])[CH2][CH2][S]', '[c][n][n][c]([n])[n]', '[CH2][CH2][S][c]([n])[n]', '[c][cH][cH][cH][cH]', '[cH][cH][cH][cH][c]', '[C][CH2][CH2][CH2][CH2]', '[c][CH2][C]([CH2][CH2])([CH2][CH2])[c]([c])[c]', '[cH][cH][c]([cH][cH])[CH2][CH2]', '[c]-[c]([n])[c]([c]([n])=[O])[C]([CH2])([CH2]

In [16]:
instance[0]

"'[cH][cH][c]([CH3])[cH][cH]', '[c]-[n]([c])[c](=[O])[c]([c])[C]', '[cH][cH][c](-[c]([c])[n])[c]([cH])[CH2]', '[c][n]([c])[c]([n][n])[S][CH2]', '[cH][cH][c]([cH][cH])-[n]([c])[c]', '[cH][cH][cH][c]([c])[CH2]', '[c][C]([CH2])([CH2])[CH2][c]([c])[cH]', '[c][cH][cH][c]([cH])-[n]', '[c][C]([CH2])([CH2])[CH2][CH2][CH2]', '[cH][c]([cH])-[n]([c]([c])=[O])[c]([n])[n]', '[c]-[c]([c])[n]([c]([n])[S])[c]([n])[n]', '[cH][c]([c])-[c]([c]([c])[C])[n]([c])[c]', '[c]-[c]([c])[cH][cH][cH]', '[c]-[c]([cH])[c]([cH][cH])[CH2][C]', '[c]-[n]([c])[c]([n][n])[n]([c])[c]', '[CH2][CH2][CH2][CH2][CH2]', '[cH][cH][cH][c]([cH])[CH2]', '[c][CH2][CH2][S][c]', '[c][cH][cH][c]([cH])[CH3]', '[c][n][n][c]([n])[S]', '[cH][cH][cH][cH][cH]', '[cH][c]([cH])[CH2][CH2][S]', '[c][n][n][c]([n])[n]', '[CH2][CH2][S][c]([n])[n]', '[c][cH][cH][cH][cH]', '[cH][cH][cH][cH][c]', '[C][CH2][CH2][CH2][CH2]', '[c][CH2][C]([CH2][CH2])([CH2][CH2])[c]([c])[c]', '[cH][cH][c]([cH][cH])[CH2][CH2]', '[c]-[c]([n])[c]([c]([n])=[O])[C]([CH2])([CH2]

In [18]:
suppl_csv = suppl_csv [['Fragments']]
suppl_csv

,Fragments
0,"'[cH][cH][c]([CH3])[cH][cH]', '[c]-[n]([c])[c]..."
1,"'[cH][cH][c]([CH3])[cH][cH]', '[c][n][n][c]([C..."
2,"'[O][CH2][CH2][NH][S]', '[C][CH]([CH2])[CH2][C..."
3,"'[cH][c]([CH2])[cH][c]([cH])[Br]', '[cH][cH][c..."
4,'[cH][c]([cH])-[c]([c]([CH3])[s])[n]([c])[CH2]...
...,...
379007,"'[c][c]([cH])[O][CH3]', '[CH][c]([cH])[cH][c](..."
379008,"'[C][NH][CH2][c]([c])[cH]', '[CH3][CH2][CH2][C..."
379009,"'[c][cH][c]([n][c])[CH]([CH2])[CH2]', '[CH][CH..."
379010,"'[c][CH]([NH])[c]([cH][cH])[cH][cH]', '[c][C](..."


In [19]:
df_split = np.array_split(suppl_csv, 10)

### Coocurence profile #0

In [20]:
df_split[0] = df_split[0] [['Fragments']]

In [21]:
df_split[0]

,Fragments
0,"'[cH][cH][c]([CH3])[cH][cH]', '[c]-[n]([c])[c]..."
1,"'[cH][cH][c]([CH3])[cH][cH]', '[c][n][n][c]([C..."
2,"'[O][CH2][CH2][NH][S]', '[C][CH]([CH2])[CH2][C..."
3,"'[cH][c]([CH2])[cH][c]([cH])[Br]', '[cH][cH][c..."
4,'[cH][c]([cH])-[c]([c]([CH3])[s])[n]([c])[CH2]...
...,...
37897,"'[cH][cH][c]([cH][cH])[S]([N])(=[O])=[O]', '[C..."
37898,"'[CH2][CH2][CH]([CH2][N])[C]([NH])=[O]', '[cH]..."
37899,"'[cH][cH][c]([CH3])[cH][cH]', '[CH][c]([cH])[n..."
37900,"'[cH][cH][c]([O][CH])[c]([c])[n]', '[CH3][C]([..."


In [22]:
feature_list0 = []
for x in df_split[0]['Fragments']:
    tup = literal_eval(x)
    feature_list0.append(tup)
feature_list0[-1]

('[CH2][CH2][CH]([CH2][N])[C]([NH])=[O]',
 '[cH][cH][s][c]([cH])[S]',
 '[CH][CH2][CH2][CH2][N]',
 '[CH2][CH2][CH2][N]([CH2])[S]',
 '[c][CH]([NH])[CH2][CH3]',
 '[cH][cH][c]([s][cH])[S]([N])(=[O])=[O]',
 '[c][CH]([CH2])[NH][C]([CH])=[O]',
 '[cH][c]([CH])[cH][c]([c])[CH3]',
 '[CH][NH][C](=[O])[CH]([CH2])[CH2]',
 '[cH][cH][cH][s][c]',
 '[c][cH][cH][cH][s]',
 '[c][cH][cH][c]([cH])[CH]',
 '[c][cH][c]([CH3])[c]([cH])[CH3]',
 '[cH][cH][cH][c]([s])[S]',
 '[C][CH]([CH2])[CH2][N]([CH2])[S]',
 '[C][CH]([CH2])[CH2][CH2][CH2]',
 '[cH][c]([s])[S](=[O])(=[O])[N]([CH2])[CH2]',
 '[cH][cH][c]([CH3])[c]([cH])[CH3]',
 '[c][cH][cH][c]([c])[CH3]',
 '[c][S](=[O])(=[O])[N]([CH2][CH])[CH2][CH2]',
 '[cH][cH][c]([cH][c])[CH]([CH2])[NH]',
 '[C][NH][CH]([CH2][CH3])[c]([cH])[cH]')

In [23]:
feature_list0[0]

('[cH][cH][c]([CH3])[cH][cH]',
 '[c]-[n]([c])[c](=[O])[c]([c])[C]',
 '[cH][cH][c](-[c]([c])[n])[c]([cH])[CH2]',
 '[c][n]([c])[c]([n][n])[S][CH2]',
 '[cH][cH][c]([cH][cH])-[n]([c])[c]',
 '[cH][cH][cH][c]([c])[CH2]',
 '[c][C]([CH2])([CH2])[CH2][c]([c])[cH]',
 '[c][cH][cH][c]([cH])-[n]',
 '[c][C]([CH2])([CH2])[CH2][CH2][CH2]',
 '[cH][c]([cH])-[n]([c]([c])=[O])[c]([n])[n]',
 '[c]-[c]([c])[n]([c]([n])[S])[c]([n])[n]',
 '[cH][c]([c])-[c]([c]([c])[C])[n]([c])[c]',
 '[c]-[c]([c])[cH][cH][cH]',
 '[c]-[c]([cH])[c]([cH][cH])[CH2][C]',
 '[c]-[n]([c])[c]([n][n])[n]([c])[c]',
 '[CH2][CH2][CH2][CH2][CH2]',
 '[cH][cH][cH][c]([cH])[CH2]',
 '[c][CH2][CH2][S][c]',
 '[c][cH][cH][c]([cH])[CH3]',
 '[c][n][n][c]([n])[S]',
 '[cH][cH][cH][cH][cH]',
 '[cH][c]([cH])[CH2][CH2][S]',
 '[c][n][n][c]([n])[n]',
 '[CH2][CH2][S][c]([n])[n]',
 '[c][cH][cH][cH][cH]',
 '[cH][cH][cH][cH][c]',
 '[C][CH2][CH2][CH2][CH2]',
 '[c][CH2][C]([CH2][CH2])([CH2][CH2])[c]([c])[c]',
 '[cH][cH][c]([cH][cH])[CH2][CH2]',
 '[c]-[c]([n])[c](

In [24]:
#Subset #0
p0 = CooccurrenceProfile.from_feature_lists(feature_list0)

In [25]:
p0.df

value
feature1                                   feature2                                         
[CH2][CH2][CH2][CH2][CH2]                  [CH2][CH2][CH2][CH2][CH2]                    1906
                                           [CH2][CH2][S][c]([n])[n]                        9
                                           [C][CH2][CH2][CH2][CH2]                       330
                                           [cH][cH][cH][cH][cH]                          263
                                           [cH][cH][cH][cH][c]                           392
...                                                                                      ...
[cH][cH][s][c]([cH])[S]                    [c][cH][cH][c]([c])[CH3]                        1
[cH][c]([CH])[cH][c]([c])[CH3]             [cH][c]([s])[S](=[O])(=[O])[N]([CH2])[CH2]      1
                                           [c][S](=[O])(=[O])[N]([CH2][CH])[CH2][CH2]      1
[cH][c]([s])[S](=[O])(=[O])[N]([CH2])[CH2] [c][cH][cH][c]([c])[CH3]                        1
[c][S](=[O])(=[O])[N]([CH2][CH])[CH2][CH2] [c][cH][c]([CH3])[c]([cH])[CH3]                 1

[3089246 rows x 1 columns]

In [26]:
p0.df.to_csv('../data/CPF/ZINC/ZINCp0_37902.csv')

In [27]:
p0 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/ZINC/ZINCp0_37902.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37902)
p0.df

value
feature1                                   feature2                                         
[CH2][CH2][CH2][CH2][CH2]                  [CH2][CH2][CH2][CH2][CH2]                    1906
                                           [CH2][CH2][S][c]([n])[n]                        9
                                           [C][CH2][CH2][CH2][CH2]                       330
                                           [cH][cH][cH][cH][cH]                          263
                                           [cH][cH][cH][cH][c]                           392
...                                                                                      ...
[cH][cH][s][c]([cH])[S]                    [c][cH][cH][c]([c])[CH3]                        1
[cH][c]([CH])[cH][c]([c])[CH3]             [cH][c]([s])[S](=[O])(=[O])[N]([CH2])[CH2]      1
                                           [c][S](=[O])(=[O])[N]([CH2][CH])[CH2][CH2]      1
[cH][c]([s])[S](=[O])(=[O])[N]([CH2])[CH2] [c][cH][cH][c]([c])[CH3]                        1
[c][S](=[O])(=[O])[N]([CH2][CH])[CH2][CH2] [c][cH][c]([CH3])[c]([cH])[CH3]                 1

[3089246 rows x 1 columns]

### Coocurence profile #1

In [28]:
df_split[1] = df_split[1] [['Fragments']]
df_split[1]

,Fragments
37902,"'[c][cH][c]([n][c])-[c]([cH])[o]', '[c][c]([s]..."
37903,"'[cH][cH][cH][c]([c])[N]', '[cH][cH][c]([c]([c..."
37904,"'[cH][cH][cH][c]([c])[s]', '[cH][c]([c])[CH]=[..."
37905,"'[cH][cH][c]([F])[cH][cH]', '[c][NH][C](=[O])[..."
37906,"'[cH][cH][c]([cH][cH])[C]([CH])=[O]', '[c][cH]..."
...,...
75799,"'[cH][cH][c]([CH3])[cH][cH]', '[CH]=[N+]([CH])..."
75800,"'[c][cH][cH][n][cH]', '[c][cH][cH][cH][c]', '[..."
75801,"'[N][CH2][CH]1[CH2][CH2]1', '[N]#[C][c]([n])[n..."
75802,"'[cH][cH][cH][c]([c])[N]', '[c][O][CH]([CH3])[..."


In [29]:
feature_list1 = []
for x in df_split[1]['Fragments']:
    tup = literal_eval(x)
    feature_list1.append(tup)
feature_list1[-1]

('[cH][cH][c]([n][c])[C](=[O])[OH]',
 '[C][c]([n])[cH][cH][c]',
 '[C][c]([cH])[n][c]([c])[O]',
 '[c][c]([n])[O][CH3]',
 '[c][n][c]([O][CH3])[c]([cH])[F]',
 '[cH][c]([n])[C](=[O])[OH]',
 '[c][cH][cH][c]([c])[F]',
 '[cH][cH][c]([F])[c]([n])[O]')

In [30]:
feature_list1[0]

('[c][cH][c]([n][c])-[c]([cH])[o]',
 '[c][c]([s])[CH2][CH2][CH2]',
 '[C][c]([cH])[c]([n][c])[n]([c])[n]',
 '[c][cH][c]([n]([c])[n])[C]([F])([F])[F]',
 '[C][c]([c])[cH][n][n]',
 '[c][c]([c])[C](=[O])[O][CH2]',
 '[cH][cH][cH][o][c]',
 '[c]-[c]([cH])[n][c]([c])[n]',
 '[cH][c]([n])[C]([F])([F])[F]',
 '[c][C](=[O])[NH][c]([c])[s]',
 '[c][c]([CH2])[s][c]([c])[NH]',
 '[c]-[c]([cH])[o][cH][cH]',
 '[c][s][c]([CH2][CH2])[c]([c])[CH2]',
 '[C][c]([c])[c]([CH2][CH2])[c]([CH2])[s]',
 '[c][c]([c])[CH2][CH2][CH2]',
 '[c][NH][C](=[O])[c]([cH])[c]',
 '[c][c]([CH2])[c]([C](=[O])[O])[c]([NH])[s]',
 '[c][cH][n][n]([c])[c]',
 '[c]-[c]([o])[cH][cH][cH]',
 '[c][cH][cH][cH][o]',
 '[c]-[c]([n])[cH][c]([C])[n]',
 '[n][cH][c]([C]([NH])=[O])[c]([n])[n]',
 '[cH][cH][c]([o][cH])-[c]([cH])[n]',
 '[C][NH][c]([s][c])[c]([C])[c]',
 '[cH][n][n]([c]([cH])[C])[c]([c])[n]',
 '[C][O][CH2][CH3]',
 '[c][CH2][CH2][CH2][CH2]',
 '[c][C](=[O])[O][CH2][CH3]')

In [31]:
p1 = CooccurrenceProfile.from_feature_lists(feature_list1)
p1.df

value
feature1                        feature2                                      
[C][NH][c]([s][c])[c]([C])[c]   [C][NH][c]([s][c])[c]([C])[c]              319
                                [C][O][CH2][CH3]                           144
                                [C][c]([cH])[c]([n][c])[n]([c])[n]           4
                                [C][c]([c])[cH][n][n]                        5
                                [C][c]([c])[c]([CH2][CH2])[c]([CH2])[s]    217
...                                                                        ...
[cH][c]([n])[C](=[O])[OH]       [c][c]([n])[O][CH3]                          1
                                [c][n][c]([O][CH3])[c]([cH])[F]              1
[c][cH][cH][c]([c])[F]          [c][n][c]([O][CH3])[c]([cH])[F]              1
[c][c]([n])[O][CH3]             [c][n][c]([O][CH3])[c]([cH])[F]              1
[c][n][c]([O][CH3])[c]([cH])[F] [c][n][c]([O][CH3])[c]([cH])[F]              1

[3090440 rows x 1 columns]

In [32]:
p1.df.to_csv('../data/CPF/ZINC/ZINCp1_37902.csv')

In [33]:
p1 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/ZINC/ZINCp1_37902.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37902)
p1.df

value
feature1                        feature2                                      
[C][NH][c]([s][c])[c]([C])[c]   [C][NH][c]([s][c])[c]([C])[c]              319
                                [C][O][CH2][CH3]                           144
                                [C][c]([cH])[c]([n][c])[n]([c])[n]           4
                                [C][c]([c])[cH][n][n]                        5
                                [C][c]([c])[c]([CH2][CH2])[c]([CH2])[s]    217
...                                                                        ...
[cH][c]([n])[C](=[O])[OH]       [c][c]([n])[O][CH3]                          1
                                [c][n][c]([O][CH3])[c]([cH])[F]              1
[c][cH][cH][c]([c])[F]          [c][n][c]([O][CH3])[c]([cH])[F]              1
[c][c]([n])[O][CH3]             [c][n][c]([O][CH3])[c]([cH])[F]              1
[c][n][c]([O][CH3])[c]([cH])[F] [c][n][c]([O][CH3])[c]([cH])[F]              1

[3090440 rows x 1 columns]

### Coocurence profile 2

In [34]:
df_split[2] = df_split[2] [['Fragments']]
df_split[2]

,Fragments
75804,"'[c]-[c]([cH])[cH][cH][c]', '[cH][c]([cH])[O][..."
75805,"'[C][CH]([CH])[C]([CH3])([CH]=[CH])[O][C]', '[..."
75806,"'[CH2][CH2][N]([CH2][CH2])[C]([CH3])=[O]', '[C..."
75807,"'[c][S](=[O])(=[O])[NH][c]([cH])[c]', '[cH][c]..."
75808,"'[cH][cH][cH][c]([c])[CH3]', '[c][n][c]([n][nH..."
...,...
113700,"'[CH2][CH2][CH2][CH2][N]', '[c]-[c]([cH])[cH][..."
113701,"'[CH2][CH2][NH][C]([C])=[O]', '[C][C]([CH])([C..."
113702,"'[c][S](=[O])(=[O])[N]([CH2][CH2])[CH2][CH2]',..."
113703,"'[CH][c]([cH])[cH][c]([c])[O]', '[c][CH]([n])[..."


In [35]:
feature_list2 = []
for x in df_split[2]['Fragments']:
    tup = literal_eval(x)
    feature_list2.append(tup)
feature_list2[-1]

('[C][CH]([CH])[CH2][CH]([CH2])[C]',
 '[C][c]([cH])[c]([cH][cH])[NH][C]',
 '[CH2][CH2][CH2][CH]([CH3])[N]',
 '[CH][CH2][CH]([CH2][CH2])[C]([CH2])([NH])[NH]',
 '[CH][CH2][CH2][CH]([CH])[CH2]',
 '[c][c]([cH])[NH][C]([CH])([CH2])[NH]',
 '[C][CH]([CH2])[CH]([CH2][CH2])[CH2][C]',
 '[cH][cH][cH][c]([c])[NH]',
 '[CH][CH2][CH2][CH]([CH2])[C]',
 '[CH][CH]([CH2])[CH2][C]([CH])([NH])[NH]',
 '[cH][c]([c])[C](=[O])[NH][C]',
 '[C][c]([c])[cH][cH][cH]',
 '[C][NH][CH2][CH2][N]',
 '[CH2][CH2][CH]([CH3])[N]([CH])[CH2]',
 '[CH2][NH][C](=[O])[CH]([CH2])[CH]',
 '[CH][N]([CH])[CH2][CH2][NH]',
 '[CH][CH2][CH]([C]([NH])=[O])[CH]([CH2])[CH2]',
 '[CH][CH2][CH2][CH2][CH]',
 '[C][NH][C]([CH2][CH])([NH][c])[CH]([CH2])[CH2]',
 '[cH][cH][c]([C]([NH])=[O])[c]([cH])[NH]',
 '[c][cH][cH][cH][cH]',
 '[cH][cH][cH][cH][c]',
 '[CH2][CH2][NH][C]([CH])=[O]',
 '[CH2][CH2][N]([CH]([CH3])[CH2])[CH]([CH2])[CH3]',
 '[c][C](=[O])[NH][C]([CH])([CH2])[NH]')

In [36]:
feature_list2[0]

('[c]-[c]([cH])[cH][cH][c]',
 '[cH][c]([cH])[O][CH2][CH]',
 '[cH][cH][c]([cH][cH])-[c]([cH])[cH]',
 '[cH][c]([cH])[CH]([OH])[CH2][O]',
 '[cH][c]([cH])[C]#[N]',
 '[C][c]([cH])[cH][cH][c]',
 '[cH][cH][c]([cH][cH])[CH]([CH2])[OH]',
 '[c][cH][cH][c]([cH])[O]',
 '[cH][cH][c]([C]#[N])[cH][cH]',
 '[cH][cH][c]([cH][cH])[O][CH2]',
 '[c][O][CH2][CH]([c])[OH]',
 '[c][cH][cH][c]([cH])[CH]')

In [37]:
p2 = CooccurrenceProfile.from_feature_lists(feature_list2)
p2.df

value
feature1                feature2                                      
[C][c]([cH])[cH][cH][c] [C][c]([cH])[cH][cH][c]                   4259
                        [cH][cH][c]([C]#[N])[cH][cH]               136
                        [cH][cH][c]([cH][cH])-[c]([cH])[cH]         67
                        [cH][cH][c]([cH][cH])[CH]([CH2])[OH]         1
                        [cH][cH][c]([cH][cH])[O][CH2]              303
...                                                                ...
[C][NH][CH2][CH2][N]    [C][c]([cH])[c]([cH][cH])[NH][C]             1
                        [cH][cH][c]([C]([NH])=[O])[c]([cH])[NH]      1
                        [cH][c]([c])[C](=[O])[NH][C]                 1
                        [c][C](=[O])[NH][C]([CH])([CH2])[NH]         1
                        [c][c]([cH])[NH][C]([CH])([CH2])[NH]         1

[3092274 rows x 1 columns]

In [38]:
p2.df.to_csv('../data/CPF/ZINC/ZINCp2_37901.csv')

In [39]:
p2 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/ZINC/ZINCp2_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p2.df

value
feature1                feature2                                      
[C][c]([cH])[cH][cH][c] [C][c]([cH])[cH][cH][c]                   4259
                        [cH][cH][c]([C]#[N])[cH][cH]               136
                        [cH][cH][c]([cH][cH])-[c]([cH])[cH]         67
                        [cH][cH][c]([cH][cH])[CH]([CH2])[OH]         1
                        [cH][cH][c]([cH][cH])[O][CH2]              303
...                                                                ...
[C][NH][CH2][CH2][N]    [C][c]([cH])[c]([cH][cH])[NH][C]             1
                        [cH][cH][c]([C]([NH])=[O])[c]([cH])[NH]      1
                        [cH][c]([c])[C](=[O])[NH][C]                 1
                        [c][C](=[O])[NH][C]([CH])([CH2])[NH]         1
                        [c][c]([cH])[NH][C]([CH])([CH2])[NH]         1

[3092274 rows x 1 columns]

### Coocurence profile 3

In [40]:
df_split[3] = df_split[3] [['Fragments']]
df_split[3]

,Fragments
113705,"'[CH2][CH2][CH2][CH2][CH2]', '[CH][CH]([CH2])[..."
113706,"'[c][cH][cH][n][cH]', '[cH][cH][c]([cH][cH])-[..."
113707,"'[C][c]([cH])[cH][cH][cH]', '[cH][cH][o][c]([c..."
113708,"'[cH][cH][cH][c]([c])[N]', '[cH][cH][c]([c]([c..."
113709,"'[c][s][c]([CH2][CH2])[n][n]', '[cH][cH][c]([c..."
...,...
151601,"'[CH2][CH2][CH2][CH2][N]', '[C]=[CH][c]([c]([n..."
151602,"'[cH][cH][c]([CH3])[cH][cH]', '[cH][c]([cH])[C..."
151603,"'[CH2][CH2][CH2][O][CH]', '[CH2][CH2][CH2][CH]..."
151604,"'[CH][NH][C](=[O])[CH]([CH3])[N]', '[C][N]([CH..."


In [41]:
feature_list3 = []
for x in df_split[3]['Fragments']:
    tup = literal_eval(x)
    feature_list3.append(tup)
feature_list3[-1]

('[c][NH][C]([CH3])=[O]',
 '[CH2][CH2][CH]([S][CH3])[CH]([CH2])[NH]',
 '[c][C](=[O])[NH][CH]([CH])[CH2]',
 '[cH][c]([cH])[C](=[O])[NH][CH]',
 '[c][cH][cH][c]([cH])[NH]',
 '[CH3][S][CH]([CH2])[CH]',
 '[CH2][CH2][CH2][CH]([CH])[S]',
 '[cH][cH][c]([cH][cH])[C]([NH])=[O]',
 '[C][NH][c]([cH][cH])[cH][cH]',
 '[C][c]([cH])[cH][cH][c]',
 '[CH2][CH2][CH2][CH]([CH])[NH]',
 '[C][NH][CH]([CH2][CH2])[CH]([CH2])[S]',
 '[CH2][CH2][CH2][CH2][CH]',
 '[cH][c]([cH])[NH][C]([CH3])=[O]',
 '[CH][CH2][CH2][CH2][CH2]')

In [42]:
feature_list3[0]

('[CH2][CH2][CH2][CH2][CH2]',
 '[CH][CH]([CH2])[CH]([CH2][CH])[C](=[O])[O]',
 '[CH]=[CH][CH]([CH2][CH])[CH2][CH]',
 '[CH][CH]([CH2])[CH2][CH]([CH])[CH]',
 '[CH][CH]=[CH][CH]([CH])[CH2]',
 '[CH][C](=[O])[O][CH]([CH2])[CH2]',
 '[CH][CH]=[CH][CH]([CH2])[CH2]',
 '[CH2][CH2][CH2][CH]([CH2])[O]',
 '[C][O][CH]([CH2][CH2])[CH2][CH2]',
 '[CH][O][C](=[O])[CH]([CH2])[CH]',
 '[CH][CH2][CH2][CH2][CH2]',
 '[C][CH]([CH2])[CH]([CH]=[CH])[CH2][CH]',
 '[C][CH]([CH])[CH2][CH]([CH])[CH2]')

In [43]:
p3 = CooccurrenceProfile.from_feature_lists(feature_list3)
p3.df

value
feature1                              feature2                                 
[CH2][CH2][CH2][CH2][CH2]             [CH2][CH2][CH2][CH2][CH2]            2013
                                      [CH2][CH2][CH2][CH]([CH2])[O]          26
                                      [CH]=[CH][CH]([CH2][CH])[CH2][CH]       1
                                      [CH][CH2][CH2][CH2][CH2]              668
                                      [CH][CH]([CH2])[CH2][CH]([CH])[CH]      1
...                                                                         ...
[C][NH][CH]([CH2][CH2])[CH]([CH2])[S] [c][C](=[O])[NH][CH]([CH])[CH2]         1
                                      [c][NH][C]([CH3])=[O]                   1
                                      [c][cH][cH][c]([cH])[NH]                1
[cH][c]([cH])[NH][C]([CH3])=[O]       [c][C](=[O])[NH][CH]([CH])[CH2]         1
[c][C](=[O])[NH][CH]([CH])[CH2]       [c][NH][C]([CH3])=[O]                   1

[3112470 rows x 1 columns]

In [44]:
p3.df.to_csv('../data/CPF/ZINC/ZINCp3_37901.csv')

In [45]:
p3 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/ZINC/ZINCp3_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p3.df

value
feature1                              feature2                                 
[CH2][CH2][CH2][CH2][CH2]             [CH2][CH2][CH2][CH2][CH2]            2013
                                      [CH2][CH2][CH2][CH]([CH2])[O]          26
                                      [CH]=[CH][CH]([CH2][CH])[CH2][CH]       1
                                      [CH][CH2][CH2][CH2][CH2]              668
                                      [CH][CH]([CH2])[CH2][CH]([CH])[CH]      1
...                                                                         ...
[C][NH][CH]([CH2][CH2])[CH]([CH2])[S] [c][C](=[O])[NH][CH]([CH])[CH2]         1
                                      [c][NH][C]([CH3])=[O]                   1
                                      [c][cH][cH][c]([cH])[NH]                1
[cH][c]([cH])[NH][C]([CH3])=[O]       [c][C](=[O])[NH][CH]([CH])[CH2]         1
[c][C](=[O])[NH][CH]([CH])[CH2]       [c][NH][C]([CH3])=[O]                   1

[3112470 rows x 1 columns]

### Coocurence profile 4

In [46]:
df_split[4] = df_split[4] [['Fragments']]
df_split[4]

,Fragments
151606,"'[C][c]([c])[cH][c]([CH3])[s]', '[CH2][CH2][CH..."
151607,"'[cH][cH][c]([CH3])[cH][cH]', '[c]-[c]([n])[n]..."
151608,"'[CH2][CH2][CH2][C]([CH2])([CH2])[O]', '[C][c]..."
151609,"'[CH][CH2][CH2][CH2][N]', '[C][c]([cH])[n]([CH..."
151610,"'[cH][cH][c]([CH3])[cH][cH]', '[c][n][n][c]([C..."
...,...
189502,"'[c][CH2][C](=[O])[NH][CH]', '[cH][cH][cH][c](..."
189503,"'[C][S][CH]=[C]([c])[N]', '[C][c]([cH])[cH][cH..."
189504,"'[c]=[N][c]([cH][cH])[cH][cH]', '[cH][cH][cH][..."
189505,"'[cH][cH][c]([cH][cH])[S]([N])(=[O])=[O]', '[c..."


In [47]:
feature_list4 = []
for x in df_split[4]['Fragments']:
    tup = literal_eval(x)
    feature_list4.append(tup)
feature_list4[-1]

('[cH][cH][cH][c]([c])[n]',
 '[cH][cH][cH][c]([c])[c]',
 '[c][n][cH][n]([c])[CH2]',
 '[c][c]([cH])[c](=[O])[n]([cH])[CH2]',
 '[CH3][N]([CH3])[S](=[O])(=[O])[N]([CH2])[CH2]',
 '[cH][cH][c]([c]([n])=[O])[c]([cH])[n]',
 '[cH][n]([c])[CH2][C]([N])=[O]',
 '[C][N]([CH2])[CH2][CH2][N]',
 '[CH3][N]([CH3])[S]([N])(=[O])=[O]',
 '[CH2][N]([CH2])[C](=[O])[CH2][n]',
 '[CH2][N]([S])[CH2][CH2][N]',
 '[CH2][CH2][N]([CH2][CH2])[C]([CH2])=[O]',
 '[CH2][CH2][N]([CH2][CH2])[S]([N])(=[O])=[O]',
 '[c][cH][cH][cH][cH]',
 '[cH][cH][cH][cH][c]',
 '[cH][c]([c])[n][cH][n]',
 '[cH][cH][c]([n][cH])[c]([cH])[c]',
 '[C][CH2][n]([cH][n])[c]([c])=[O]')

In [48]:
feature_list4[0]

('[C][c]([c])[cH][c]([CH3])[s]',
 '[CH2][CH2][CH2][CH2][N]',
 '[c][NH][C](=[O])[CH2][CH2]',
 '[c][S](=[O])(=[O])[N]([CH2][CH2])[CH2][CH2]',
 '[C][CH2][CH2][C]([NH])=[O]',
 '[c][c]([NH])[cH][c]([cH])[S]',
 '[CH2][CH2][CH2][N]([CH2])[S]',
 '[c][cH][cH][c]([c])[O]',
 '[cH][c]([c])[C](=[O])[CH2][CH2]',
 '[cH][c]([cH])[S](=[O])(=[O])[N]([CH2])[CH2]',
 '[c][cH][c]([C]([CH2])=[O])[c]([CH3])[s]',
 '[cH][c]([c])[O][CH3]',
 '[CH2][CH2][CH2][CH2][CH2]',
 '[c][cH][c]([CH3])[s][c]',
 '[c][cH][cH][c]([cH])[S]',
 '[cH][cH][c]([O][CH3])[c]([cH])[NH]',
 '[c][c]([cH])[NH][C]([CH2])=[O]',
 '[cH][cH][c]([cH][c])[S]([N])(=[O])=[O]',
 '[C][c]([cH])[c]([CH3])[s][c]',
 '[C][NH][c]([cH][c])[c]([cH])[O]',
 '[C][CH2][CH2][C]([c])=[O]',
 '[c][c]([CH3])[s][c]([cH])[CH3]')

In [49]:
p4 = CooccurrenceProfile.from_feature_lists(feature_list4)
p4.df

value
feature1                          feature2                                  
[CH2][CH2][CH2][CH2][CH2]         [CH2][CH2][CH2][CH2][CH2]             1962
                                  [CH2][CH2][CH2][CH2][N]                829
                                  [CH2][CH2][CH2][N]([CH2])[S]           242
                                  [C][CH2][CH2][C]([NH])=[O]               8
                                  [C][CH2][CH2][C]([c])=[O]                3
...                                                                      ...
[CH3][N]([CH3])[S]([N])(=[O])=[O] [cH][c]([c])[n][cH][n]                   1
                                  [cH][n]([c])[CH2][C]([N])=[O]            1
                                  [c][c]([cH])[c](=[O])[n]([cH])[CH2]      1
                                  [c][n][cH][n]([c])[CH2]                  1
[C][N]([CH2])[CH2][CH2][N]        [cH][cH][c]([n][cH])[c]([cH])[c]         1

[3117683 rows x 1 columns]

In [50]:
p4.df.to_csv('../data/CPF/ZINC/ZINCp4_37901.csv')

In [51]:
p4 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/ZINC/ZINCp4_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p4.df

value
feature1                          feature2                                  
[CH2][CH2][CH2][CH2][CH2]         [CH2][CH2][CH2][CH2][CH2]             1962
                                  [CH2][CH2][CH2][CH2][N]                829
                                  [CH2][CH2][CH2][N]([CH2])[S]           242
                                  [C][CH2][CH2][C]([NH])=[O]               8
                                  [C][CH2][CH2][C]([c])=[O]                3
...                                                                      ...
[CH3][N]([CH3])[S]([N])(=[O])=[O] [cH][c]([c])[n][cH][n]                   1
                                  [cH][n]([c])[CH2][C]([N])=[O]            1
                                  [c][c]([cH])[c](=[O])[n]([cH])[CH2]      1
                                  [c][n][cH][n]([c])[CH2]                  1
[C][N]([CH2])[CH2][CH2][N]        [cH][cH][c]([n][cH])[c]([cH])[c]         1

[3117683 rows x 1 columns]

### Coocurence profile 5

In [52]:
df_split[5] = df_split[5] [['Fragments']]
df_split[5]

,Fragments
189507,"'[C][c]([cH])[cH][c]([c])[Br]', '[c][c]([c])[O..."
189508,"'[C][c]([cH])[c]([n][n])[S]([N])(=[O])=[O]', '..."
189509,"'[cH][cH][c]([cH][cH])[S]([N])(=[O])=[O]', '[c..."
189510,"'[c][n][cH][c]([n])[N]', '[C][N]([CH3])[CH]([C..."
189511,"'[CH2][C]([CH3])([CH2])[CH2][CH2][N]', '[CH2][..."
...,...
227403,"'[CH2][CH2][c]([cH][n])[n][n]', '[c][CH2][CH2]..."
227404,"'[cH][c]([n])[C]([CH3])([CH3])[CH3]', '[C][NH]..."
227405,"'[cH][cH][c]([CH3])[cH][cH]', '[c][n][n][c]([C..."
227406,"'[cH][c]([cH])[CH]([NH][NH2])[c]([cH])[cH]', '..."


In [53]:
feature_list5 = []
for x in df_split[5]['Fragments']:
    tup = literal_eval(x)
    feature_list5.append(tup)
feature_list5[-1]

('[c][N]=[C]([S][CH])[c]([cH])[cH]',
 '[c][O][CH2][C]([NH2])=[O]',
 '[C][c]([cH])[cH][cH][cH]',
 '[c][c]([CH])[c]([CH3])[n][n]',
 '[cH][cH][c]([cH][cH])[C](=[N])[S]',
 '[cH][cH][c]([F])[cH][cH]',
 '[c]-[n]([n])[c]([N]=[C])[c]([c])[CH]',
 '[c][cH][c]([O][CH3])[c]([cH])[O]',
 '[c][cH][cH][c]([cH])-[n]',
 '[c][cH][cH][c]([c])[O]',
 '[c][CH]([S])[c]([cH][c])[cH][cH]',
 '[cH][cH][c]([O][CH2])[c]([cH])[O]',
 '[C][CH2][O][c]([c])[cH]',
 '[c][C](=[N])[S][CH]([c])[c]',
 '[NH2][C](=[O])[CH2][O]',
 '[cH][c]([c])[O][CH3]',
 '[c][CH]([S])[c]([c]([CH3])[n])[c]([N])[n]',
 '[c]-[n]([c])[n][c]([c])[CH3]',
 '[c][cH][cH][c]([cH])[CH]',
 '[c][cH][cH][c]([cH])[F]',
 '[c][n][n](-[c]([cH])[cH])[c]([c])[N]',
 '[cH][cH][cH][cH][cH]',
 '[cH][cH][c]([cH][cH])-[n]([c])[n]',
 '[c][C]([S])=[N][c]([c])[n]',
 '[c][cH][cH][cH][cH]',
 '[C][S][CH]([c]([c])[c])[c]([cH])[cH]',
 '[cH][c]([CH])[cH][c]([c])[O]')

In [54]:
feature_list5[0]

('[C][c]([cH])[cH][c]([c])[Br]',
 '[c][c]([c])[O][CH2][CH2]',
 '[c][c]([cH])[O][CH2][CH3]',
 '[c][cH][cH][c]([cH])[NH]',
 '[c][NH][C](=[O])[c]([cH])[cH]',
 '[cH][c]([cH])[O][CH3]',
 '[c][cH][c]([cH][c])[C]([NH])=[O]',
 '[C][NH][c]([cH][cH])[cH][cH]',
 '[cH][cH][c]([cH][cH])[O][CH3]',
 '[c][cH][c]([O][CH2])[c]([c])[O]',
 '[c][O][CH2][CH3]',
 '[c][cH][c]([Br])[c]([c])[O]',
 '[c][cH][cH][c]([cH])[O]',
 '[C][c]([cH])[cH][c]([c])[O]',
 '[cH][c]([O])[c]([O][CH2])[c]([cH])[Br]',
 '[CH3][CH2][CH2][O]',
 '[c][O][CH2][CH2][CH3]',
 '[c][C](=[O])[NH][c]([cH])[cH]')

In [55]:
p5 = CooccurrenceProfile.from_feature_lists(feature_list5)
p5.df

value
feature1                         feature2                                   
[CH3][CH2][CH2][O]               [CH3][CH2][CH2][O]                      236
                                 [C][NH][c]([cH][cH])[cH][cH]             39
                                 [C][c]([cH])[cH][c]([c])[Br]              5
                                 [C][c]([cH])[cH][c]([c])[O]              12
                                 [cH][cH][c]([cH][cH])[O][CH3]             9
...                                                                      ...
[c][N]=[C]([S][CH])[c]([cH])[cH] [c][c]([CH])[c]([CH3])[n][n]              1
                                 [c][n][n](-[c]([cH])[cH])[c]([c])[N]      1
[c][O][CH2][C]([NH2])=[O]        [c][cH][cH][c]([cH])-[n]                  1
                                 [c][c]([CH])[c]([CH3])[n][n]              1
                                 [c][n][n](-[c]([cH])[cH])[c]([c])[N]      1

[3102530 rows x 1 columns]

In [56]:
p5.df.to_csv('../data/CPF/ZINC/ZINCp5_37901.csv')

In [57]:
p5 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/ZINC/ZINCp5_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p5.df

value
feature1                         feature2                                   
[CH3][CH2][CH2][O]               [CH3][CH2][CH2][O]                      236
                                 [C][NH][c]([cH][cH])[cH][cH]             39
                                 [C][c]([cH])[cH][c]([c])[Br]              5
                                 [C][c]([cH])[cH][c]([c])[O]              12
                                 [cH][cH][c]([cH][cH])[O][CH3]             9
...                                                                      ...
[c][N]=[C]([S][CH])[c]([cH])[cH] [c][c]([CH])[c]([CH3])[n][n]              1
                                 [c][n][n](-[c]([cH])[cH])[c]([c])[N]      1
[c][O][CH2][C]([NH2])=[O]        [c][cH][cH][c]([cH])-[n]                  1
                                 [c][c]([CH])[c]([CH3])[n][n]              1
                                 [c][n][n](-[c]([cH])[cH])[c]([c])[N]      1

[3102530 rows x 1 columns]

### Coocurence profile 6

In [58]:
df_split[6] = df_split[6] [['Fragments']]
df_split[6]

,Fragments
227408,"'[C][CH]([CH3])[O][c]([cH])[c]', '[cH][cH][cH]..."
227409,"'[C][CH]([CH3])[O][C]([CH2])=[O]', '[C][O][CH]..."
227410,"'[cH][cH][cH][c]([c])[N]', '[cH][cH][c]([cH][c..."
227411,"'[c]-[c]([n])[cH][n]([CH2])[n]', '[c][N]([CH2]..."
227412,"'[CH2][CH2][CH2][CH2][N]', '[C][c]([cH])[cH][c..."
...,...
265304,"'[cH][cH][c]([CH3])[cH][cH]', '[cH][cH][c]([cH..."
265305,"'[C][CH2][c]([cH][cH])[cH][cH]', '[c][cH][cH][..."
265306,"'[c]-[n]([c])[c]([cH][n])-[c]([cH])[cH]', '[CH..."
265307,"'[CH2][CH2][N]([CH2][CH])[C]([CH3])=[O]', '[CH..."


In [59]:
feature_list6 = []
for x in df_split[6]['Fragments']:
    tup = literal_eval(x)
    feature_list6.append(tup)
feature_list6[-1]

('[cH][cH][c]([cH][cH])[S]([N])(=[O])=[O]',
 '[CH2][CH]([OH])[C]([OH])([CH2][N])[CH2][N]',
 '[c][S](=[O])(=[O])[N]([CH2][C])[CH2][CH]',
 '[cH][cH][cH][c]([cH])[S]',
 '[CH2][N]([S])[CH2][C]([CH2])([CH])[OH]',
 '[C][CH2][N]1[CH2][CH][CH2]1',
 '[c][CH]1[CH2][N]([CH2])[CH2]1',
 '[C][CH]([OH])[CH2][N]([CH2])[S]',
 '[CH2][N]([CH2])[CH2][C]([CH2])([CH])[OH]',
 '[c][n][n]([CH3])[c]([c])[n]',
 '[cH][c]([cH])[S](=[O])(=[O])[N]([CH2])[CH2]',
 '[c][c]([n])[CH]1[CH2][N][CH2]1',
 '[cH][n][c]([c]([CH])[n])[c]([n])[n]',
 '[cH][cH][n][c]([c])[c]',
 '[c][c]([CH])[n][n]([c])[CH3]',
 '[c][c]([n])[c]([n][n])[CH]([CH2])[CH2]',
 '[CH2][C]([CH2])([OH])[CH]([OH])[CH2][N]',
 '[cH][cH][cH][cH][cH]',
 '[cH][cH][n][c]([c])[n]',
 '[c][cH][cH][cH][cH]',
 '[cH][n][c]([c]([c])[n])[n]([CH3])[n]',
 '[c][n][cH][cH][n]')

In [60]:
feature_list6[0]

('[C][CH]([CH3])[O][c]([cH])[c]',
 '[cH][cH][cH][c]([c])[NH]',
 '[cH][c]([c])[C](=[O])[O][CH2]',
 '[C][NH][c]([cH][cH])[c]([c])[CH3]',
 '[C][c]([c])[cH][cH][cH]',
 '[c][C](=[O])[O][CH2][CH3]',
 '[cH][cH][c]([O][CH])[c]([cH])[O]',
 '[c][cH][cH][cH][c]',
 '[C][c]([cH])[c]([CH3])[c]([cH])[NH]',
 '[cH][c]([c])[NH][C]([CH])=[O]',
 '[c][O][CH]([CH3])[C]([NH])=[O]',
 '[cH][cH][cH][c]([c])[O]',
 '[c][cH][cH][cH][cH]',
 '[C][O][CH2][CH3]',
 '[cH][cH][c]([O][CH3])[c]([cH])[O]',
 '[cH][cH][c]([C](=[O])[O])[c]([c])[CH3]',
 '[cH][cH][cH][cH][c]',
 '[c][NH][C](=[O])[CH]([CH3])[O]',
 '[c][c]([cH])[O][CH3]')

In [61]:
p6 = CooccurrenceProfile.from_feature_lists(feature_list6)
p6.df

value
feature1                       feature2                                     
[C][CH]([CH3])[O][c]([cH])[c]  [C][CH]([CH3])[O][c]([cH])[c]              64
                               [C][NH][c]([cH][cH])[c]([c])[CH3]           1
                               [C][O][CH2][CH3]                            4
                               [C][c]([cH])[c]([CH3])[c]([cH])[NH]         1
                               [C][c]([c])[cH][cH][cH]                     2
...                                                                      ...
[c][c]([CH])[n][n]([c])[CH3]   [c][c]([n])[CH]1[CH2][N][CH2]1              1
[c][c]([n])[CH]1[CH2][N][CH2]1 [c][c]([n])[CH]1[CH2][N][CH2]1              1
                               [c][c]([n])[c]([n][n])[CH]([CH2])[CH2]      1
                               [c][n][cH][cH][n]                           1
                               [c][n][n]([CH3])[c]([c])[n]                 1

[3096932 rows x 1 columns]

In [62]:
p6.df.to_csv('../data/CPF/ZINC/ZINCp6_37901.csv')

In [63]:
p6 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/ZINC/ZINCp6_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p6.df

value
feature1                       feature2                                     
[C][CH]([CH3])[O][c]([cH])[c]  [C][CH]([CH3])[O][c]([cH])[c]              64
                               [C][NH][c]([cH][cH])[c]([c])[CH3]           1
                               [C][O][CH2][CH3]                            4
                               [C][c]([cH])[c]([CH3])[c]([cH])[NH]         1
                               [C][c]([c])[cH][cH][cH]                     2
...                                                                      ...
[c][c]([CH])[n][n]([c])[CH3]   [c][c]([n])[CH]1[CH2][N][CH2]1              1
[c][c]([n])[CH]1[CH2][N][CH2]1 [c][c]([n])[CH]1[CH2][N][CH2]1              1
                               [c][c]([n])[c]([n][n])[CH]([CH2])[CH2]      1
                               [c][n][cH][cH][n]                           1
                               [c][n][n]([CH3])[c]([c])[n]                 1

[3096932 rows x 1 columns]

### Coocurence profile 7

In [64]:
df_split[7] = df_split[7] [['Fragments']]
df_split[7]

,Fragments
265309,"'[c][N]([CH2])[CH2][CH2][S]', '[CH][S][CH2][CH..."
265310,"'[C][N]([CH])[CH]=[N][NH]', '[CH][NH][N]=[CH][..."
265311,"'[c][cH][c]([NH2])[n][cH]', '[CH3][CH2][CH2][C..."
265312,"'[cH][n][c]([n][cH])[N]([CH2])[CH]', '[CH2][CH..."
265313,"'[c]-[c]([s])[n][n][c]', '[C][CH2][c]([cH][cH]..."
...,...
303205,"'[c]-[c]([cH])[n][c]([cH])[c]', '[cH][cH][c]([..."
303206,"'[cH][cH][cH][c]([c])[s]', '[c][CH2][N]([CH2][..."
303207,"'[c][c]([n])[n][c]([CH2])[n]', '[cH][cH][c]([c..."
303208,"'[c][NH][C](=[O])[c]([cH])[n]', '[C][c]([cH])[..."


In [65]:
feature_list7 = []
for x in df_split[7]['Fragments']:
    tup = literal_eval(x)
    feature_list7.append(tup)
feature_list7[-1]

('[cH][c]([cH])[CH2][CH2][CH]',
 '[c][n][c]([S][CH2])[n]([c])[c]',
 '[C][NH][CH]([CH3])[CH2][CH2]',
 '[c][n]([c])[c]([n][n])[S][CH2]',
 '[c][c]([s])[n][c]([n])[S]',
 '[CH][NH][C](=[O])[CH2][S]',
 '[cH][cH][c]([cH][cH])[CH2][CH2]',
 '[c][S][CH2][C]([NH])=[O]',
 '[c][c]([CH2])[s][c]([c])[n]',
 '[c][CH2][CH2][CH]([CH3])[NH]',
 '[c][CH2][C]([CH3])([CH3])[O][CH2]',
 '[CH3][CH2][CH2][S]',
 '[c][c]([n])[n]([c]([n])[S])[c]([n])[S]',
 '[cH][cH][cH][c]([cH])[CH2]',
 '[c][c]([c])[c]([CH2][C])[c]([CH2])[s]',
 '[c][c]([CH2])[c]([c]([n])[n])[c]([n])[s]',
 '[c][n][c]([s][c])[c]([c])[c]',
 '[c][c]([c])[c]([n][n])[n]([c])[c]',
 '[c][n][n][c]([n])[S]',
 '[cH][cH][cH][cH][cH]',
 '[c][n][n][c]([c])[n]',
 '[c][c]([s])[CH2][O][C]',
 '[CH2][CH2][S][c]([n])[n]',
 '[c][cH][cH][cH][cH]',
 '[c][s][c]([CH2][O])[c]([c])[CH2]',
 '[C][CH2][S][c]([n])[n]',
 '[c][CH2][O][C]([CH2])([CH3])[CH3]',
 '[CH2][C](=[O])[NH][CH]([CH3])[CH2]',
 '[c][c]([c])[CH2][C]([CH3])([CH3])[O]',
 '[c][S][CH2][CH2][CH3]')

In [66]:
feature_list7[0]

('[c][N]([CH2])[CH2][CH2][S]',
 '[CH][S][CH2][CH2][N]',
 '[cH][c]([CH3])[cH][c]([n])[N]',
 '[c][cH][c]([n][n])[N]([CH2])[CH2]',
 '[c][cH][n][n][c]',
 '[CH2][CH2][S][CH]([C])[CH2]',
 '[CH2][CH]([S])[C]([F])([F])[F]',
 '[cH][c]([CH3])[cH][n][n]',
 '[cH][n][n][c]([cH])[N]',
 '[c][N]([CH2])[CH2][CH]([C])[S]',
 '[cH][c]([n])[N]([CH2][CH2])[CH2][CH]',
 '[CH2][S][CH]([CH2][N])[C]([F])([F])[F]',
 '[c][cH][c]([CH3])[cH][n]')

In [67]:
p7 = CooccurrenceProfile.from_feature_lists(feature_list7)
p7.df

value
feature1                     feature2                                     
[CH2][CH2][S][CH]([C])[CH2]  [CH2][CH2][S][CH]([C])[CH2]                 1
                             [CH2][CH]([S])[C]([F])([F])[F]              1
                             [CH2][S][CH]([CH2][N])[C]([F])([F])[F]      1
                             [CH][S][CH2][CH2][N]                        1
                             [cH][c]([CH3])[cH][c]([n])[N]               1
...                                                                    ...
[c][CH2][CH2][CH]([CH3])[NH] [c][n]([c])[c]([n][n])[S][CH2]              1
                             [c][n][c]([S][CH2])[n]([c])[c]              1
                             [c][n][c]([s][c])[c]([c])[c]                1
                             [c][n][n][c]([c])[n]                        1
                             [c][s][c]([CH2][O])[c]([c])[CH2]            1

[3111610 rows x 1 columns]

In [68]:
p7.df.to_csv('../data/CPF/ZINC/ZINCp7_37901.csv')

In [69]:
p7 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/ZINC/ZINCp7_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p7.df

value
feature1                     feature2                                     
[CH2][CH2][S][CH]([C])[CH2]  [CH2][CH2][S][CH]([C])[CH2]                 1
                             [CH2][CH]([S])[C]([F])([F])[F]              1
                             [CH2][S][CH]([CH2][N])[C]([F])([F])[F]      1
                             [CH][S][CH2][CH2][N]                        1
                             [cH][c]([CH3])[cH][c]([n])[N]               1
...                                                                    ...
[c][CH2][CH2][CH]([CH3])[NH] [c][n]([c])[c]([n][n])[S][CH2]              1
                             [c][n][c]([S][CH2])[n]([c])[c]              1
                             [c][n][c]([s][c])[c]([c])[c]                1
                             [c][n][n][c]([c])[n]                        1
                             [c][s][c]([CH2][O])[c]([c])[CH2]            1

[3111610 rows x 1 columns]

#### Coocurence profile 8

In [70]:
df_split[8] = df_split[8] [['Fragments']]
df_split[8]

,Fragments
303210,"'[c]-[c]([c])[n][o][c]', '[c][cH][c]([cH][cH])..."
303211,"'[C]=[C]([CH])[C](=[O])[O][CH2]', '[cH][cH][cH..."
303212,"'[cH][c]([cH])[CH2][CH3]', '[cH][cH][c]([cH][c..."
303213,"'[c][cH][cH][c]([cH])[F]', '[CH2][N]=[C]([NH2]..."
303214,"'[c][cH][n][nH][c]', '[CH][NH][C](=[O])[CH2][S..."
...,...
341106,"'[c][c]([s])[NH][C]([CH2])=[O]', '[c][n][c]([S..."
341107,"'[C][NH][CH]([CH2][CH])[CH2][N]', '[c][cH][c](..."
341108,"'[cH][c]([cH])[O][CH3]', '[C][c]([c])[o][c]([c..."
341109,"'[C][c]([cH])[cH][cH][cH]', '[cH][cH][cH][c]([..."


In [71]:
feature_list8 = []
for x in df_split[8]['Fragments']:
    tup = literal_eval(x)
    feature_list8.append(tup)
feature_list8[-1]

('[C][NH][CH2][CH2][O]',
 '[CH2][CH2][NH][C]([CH])=[O]',
 '[CH][N]([CH2])[C](=[O])[CH2][n]',
 '[cH][cH][c]([c]([cH])[n])[n]([CH2])[n]',
 '[cH][cH][c]([F])[cH][cH]',
 '[c][CH2][N]([C]([CH2])=[O])[CH]([C])[c]',
 '[CH2][CH2][O][CH3]',
 '[cH][c]([c])[n][n][n]',
 '[cH][cH][cH][s][c]',
 '[c][n]([n])[CH2][C]([N])=[O]',
 '[c][n][n][n]([c])[CH2]',
 '[c][cH][cH][cH][s]',
 '[c][CH]([N])[C](=[O])[NH][CH2]',
 '[c][cH][cH][c]([cH])[CH2]',
 '[cH][cH][cH][c]([CH])[s]',
 '[cH][cH][cH][c]([c])[n]',
 '[c][cH][cH][c]([cH])[F]',
 '[C][CH2][n]([n][n])[c]([c])[cH]',
 '[C][N]([CH])[CH2][c]([cH])[cH]',
 '[cH][cH][s][c]([cH])[CH]',
 '[C][CH]([N])[c]([cH][cH])[s][cH]',
 '[C][N]([CH2])[CH]([C]([NH])=[O])[c]([cH])[s]',
 '[cH][cH][c]([cH][cH])[CH2][N]',
 '[cH][cH][c]([n][n])[c]([cH])[n]',
 '[c][cH][cH][cH][cH]',
 '[cH][cH][cH][cH][c]',
 '[CH3][O][CH2][CH2][NH]')

In [72]:
feature_list8[0]

('[c]-[c]([c])[n][o][c]',
 '[c][cH][c]([cH][cH])[CH2][CH2]',
 '[c][C](=[O])[O][c]([c])[cH]',
 '[C][O][c]([cH][cH])[c]([cH])[O]',
 '[c][cH][c]([O][CH3])[c]([cH])[O]',
 '[c][n][o][c]([c])[CH3]',
 '[c][cH][cH][c]([c])[O]',
 '[c]-[c]([c])[c]([F])[cH][cH]',
 '[c][c]([n])-[c]([c]([cH])[F])[c]([cH])[Cl]',
 '[c]-[c]([n])[c]([C](=[O])[O])[c]([CH3])[o]',
 '[c][cH][cH][c]([cH])[CH2]',
 '[cH][c]([c])[O][CH3]',
 '[cH][c]([CH2])[cH][c]([c])[O]',
 '[c][O][C](=[O])[c]([c])[c]',
 '[c][cH][cH][cH][c]',
 '[CH2][CH2][C]([CH3])=[O]',
 '[c][CH2][CH2][C]([CH3])=[O]',
 '[c]-[c]([c])[c]([Cl])[cH][cH]',
 '[cH][cH][cH][c]([c])[Cl]',
 '[C][c]([c])[c]([n][o])-[c]([c])[c]',
 '[C][c]([c])[c]([CH3])[o][n]',
 '[C][CH2][CH2][c]([cH])[cH]',
 '[cH][cH][cH][c]([c])[F]')

In [73]:
p8 = CooccurrenceProfile.from_feature_lists(feature_list8)
p8.df

value
feature1                                     feature2                                 
[CH2][CH2][C]([CH3])=[O]                     [CH2][CH2][C]([CH3])=[O]                9
                                             [C][CH2][CH2][c]([cH])[cH]              1
                                             [C][O][c]([cH][cH])[c]([cH])[O]         1
                                             [C][c]([c])[c]([CH3])[o][n]             1
                                             [C][c]([c])[c]([n][o])-[c]([c])[c]      1
...                                                                                ...
[C][N]([CH2])[CH]([C]([NH])=[O])[c]([cH])[s] [cH][cH][c]([F])[cH][cH]                1
                                             [cH][cH][c]([cH][cH])[CH2][N]           1
                                             [c][cH][cH][c]([cH])[F]                 1
[C][N]([CH])[CH2][c]([cH])[cH]               [cH][cH][cH][c]([CH])[s]                1
                                             [cH][cH][s][c]([cH])[CH]                1

[3110222 rows x 1 columns]

In [74]:
p8.df.to_csv('../data/CPF/ZINC/ZINCp8_37901.csv')

In [75]:
p8 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/ZINC/ZINCp8_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p8.df

value
feature1                                     feature2                                 
[CH2][CH2][C]([CH3])=[O]                     [CH2][CH2][C]([CH3])=[O]                9
                                             [C][CH2][CH2][c]([cH])[cH]              1
                                             [C][O][c]([cH][cH])[c]([cH])[O]         1
                                             [C][c]([c])[c]([CH3])[o][n]             1
                                             [C][c]([c])[c]([n][o])-[c]([c])[c]      1
...                                                                                ...
[C][N]([CH2])[CH]([C]([NH])=[O])[c]([cH])[s] [cH][cH][c]([F])[cH][cH]                1
                                             [cH][cH][c]([cH][cH])[CH2][N]           1
                                             [c][cH][cH][c]([cH])[F]                 1
[C][N]([CH])[CH2][c]([cH])[cH]               [cH][cH][cH][c]([CH])[s]                1
                                             [cH][cH][s][c]([cH])[CH]                1

[3110222 rows x 1 columns]

### Coocurence profile 9

In [76]:
df_split[9] = df_split[9] [['Fragments']]
df_split[9]

,Fragments
341111,"'[C][N]([CH2])[CH2][CH2][C]', '[CH2][CH]([CH2]..."
341112,"'[c][NH][c]([cH][c])[c]([cH])[CH3]', '[c][C](=..."
341113,"'[CH2][O][CH2][CH2][N]', '[CH2][CH2][O][CH2][C..."
341114,"'[C][CH]([CH2])[CH2][NH2]', '[CH3][O][C]([CH])..."
341115,"'[cH][n][c]([C]([NH])=[O])[c]([cH])[Cl]', '[cH..."
...,...
379007,"'[c][c]([cH])[O][CH3]', '[CH][c]([cH])[cH][c](..."
379008,"'[C][NH][CH2][c]([c])[cH]', '[CH3][CH2][CH2][C..."
379009,"'[c][cH][c]([n][c])[CH]([CH2])[CH2]', '[CH][CH..."
379010,"'[c][CH]([NH])[c]([cH][cH])[cH][cH]', '[c][C](..."


In [78]:
feature_list9 = []
for x in df_split[9]['Fragments']:
    tup = literal_eval(x)
    feature_list9.append(tup)
feature_list9[-1]

('[CH2][CH2][S](=[O])(=[O])[CH2][NH]',
 '[CH][NH][CH2][S]([CH2])(=[O])=[O]',
 '[CH][CH2][CH2][S]([CH2])(=[O])=[O]',
 '[C][CH]([CH2])[NH][CH2][S]',
 '[CH2][CH]([NH])[C](=[O])[OH]',
 '[CH2][CH2][CH]([NH][CH2])[C](=[O])[OH]',
 '[C][CH]([NH])[CH2][CH2][S]')

In [79]:
feature_list9[0]

('[C][N]([CH2])[CH2][CH2][C]',
 '[CH2][CH]([CH2])[C](=[O])[N]([CH2])[CH2]',
 '[cH][cH][c](-[c]([cH])[cH])[c]([cH])[CH3]',
 '[C][C](=[O])[NH][CH]([CH3])[CH3]',
 '[cH][cH][c]([cH][c])-[c]([c])[cH]',
 '[C][C]([CH2])([CH2])[CH2][CH2][N]',
 '[c]-[c]([c])[cH][cH][cH]',
 '[CH2][CH2][CH]([CH2][CH2])[C]([N])=[O]',
 '[CH][NH][C](=[O])[C]([CH2])([CH2])[CH2]',
 '[cH][cH][cH][c]([c])[CH3]',
 '[c]-[c]([cH])[cH][c]([cH])[CH2]',
 '[cH][cH][cH][c]([cH])[CH2]',
 '[c][cH][cH][cH][c]',
 '[C][C]([CH2])([CH2])[CH2][c]([cH])[cH]',
 '[C][CH]([CH2])[CH2][CH2][CH2]',
 '[c]-[c]([cH])[cH][cH][cH]',
 '[CH2][CH2][N]([CH2][CH2])[C]([CH])=[O]',
 '[C][NH][CH]([CH3])[CH3]',
 '[c][CH2][C]([CH2][CH2])([CH2][CH2])[C]([NH])=[O]',
 '[c][cH][cH][cH][cH]',
 '[c][cH][c]([cH][cH])[CH2][C]',
 '[cH][cH][cH][cH][c]',
 '[CH][CH2][CH2][CH2][CH2]',
 '[c]-[c]([cH])[c]([CH3])[cH][cH]')

In [80]:
p9 = CooccurrenceProfile.from_feature_lists(feature_list9)
p9.df

value
feature1                               feature2                                       
[CH2][CH2][CH]([CH2][CH2])[C]([N])=[O] [CH2][CH2][CH]([CH2][CH2])[C]([N])=[O]      207
                                       [CH2][CH2][N]([CH2][CH2])[C]([CH])=[O]       88
                                       [CH2][CH]([CH2])[C](=[O])[N]([CH2])[CH2]    136
                                       [CH][CH2][CH2][CH2][CH2]                     41
                                       [CH][NH][C](=[O])[C]([CH2])([CH2])[CH2]       2
...                                                                                ...
[CH][CH2][CH2][S]([CH2])(=[O])=[O]     [C][CH]([NH])[CH2][CH2][S]                    1
[CH][NH][CH2][S]([CH2])(=[O])=[O]      [CH][NH][CH2][S]([CH2])(=[O])=[O]             1
                                       [C][CH]([CH2])[NH][CH2][S]                    1
                                       [C][CH]([NH])[CH2][CH2][S]                    1
[C][CH]([CH2])[NH][CH2][S]             [C][CH]([NH])[CH2][CH2][S]                    1

[3088780 rows x 1 columns]

In [81]:
p9.df.to_csv('../data/CPF/ZINC/ZINCp9_37901.csv')

In [82]:
p9 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/ZINC/ZINCp9_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p9.df

value
feature1                               feature2                                       
[CH2][CH2][CH]([CH2][CH2])[C]([N])=[O] [CH2][CH2][CH]([CH2][CH2])[C]([N])=[O]      207
                                       [CH2][CH2][N]([CH2][CH2])[C]([CH])=[O]       88
                                       [CH2][CH]([CH2])[C](=[O])[N]([CH2])[CH2]    136
                                       [CH][CH2][CH2][CH2][CH2]                     41
                                       [CH][NH][C](=[O])[C]([CH2])([CH2])[CH2]       2
...                                                                                ...
[CH][CH2][CH2][S]([CH2])(=[O])=[O]     [C][CH]([NH])[CH2][CH2][S]                    1
[CH][NH][CH2][S]([CH2])(=[O])=[O]      [CH][NH][CH2][S]([CH2])(=[O])=[O]             1
                                       [C][CH]([CH2])[NH][CH2][S]                    1
                                       [C][CH]([NH])[CH2][CH2][S]                    1
[C][CH]([CH2])[NH][CH2][S]             [C][CH]([NH])[CH2][CH2][S]                    1

[3088780 rows x 1 columns]

### Cooccurrence profile whole dataset ZINC

In [3]:
suppl_csv = pd.read_csv('../data/CPF/ZINC/ZINCrandomF.csv', delimiter = ' ')
suppl_csv

,Zinc_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset,Fragments
0,8432508,Cc1ccc(-n2c(=O)c3c(n4c(SCCc5ccccc5)nnc24)-c2cc...,532.229683,6.94842,DPUIXUVMXJXCJR-UHFFFAOYSA-N,DPUIXUVMXJXCJR,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[2, 58, 74, 80, 105, 116, 153, 233, 235, 289, ...","'[cH][cH][c]([CH3])[cH][cH]', '[c]-[n]([c])[c]..."
1,2325339,Cc1ccc(-n2c(COc3ccccc3)nnc2SCC(=O)Nc2ccc3c(c2)...,488.151826,4.65672,HVIYVAXZIXJRQC-UHFFFAOYSA-N,HVIYVAXZIXJRQC,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[41, 58, 80, 102, 116, 127, 184, 191, 233, 235...","'[cH][cH][c]([CH3])[cH][cH]', '[c][n][n][c]([C..."
2,408636482,COCCNS(=O)(=O)c1ccc(OCC(=O)N2CCC(C(N)=O)CC2)c(...,433.107434,0.36740,RAILYSNQBZZKMQ-UHFFFAOYSA-N,RAILYSNQBZZKMQ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[13, 80, 94, 111, 140, 198, 222, 279, 317, 319...","'[O][CH2][CH2][NH][S]', '[C][CH]([CH2])[CH2][C..."
3,97995285,Cc1ccc(S(=O)(=O)n2nc(NCc3cccc(Br)c3)nc2NCc2ccc...,588.978270,5.57272,FFUQFTXMHIMSHO-UHFFFAOYSA-N,FFUQFTXMHIMSHO,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 82, 92, 157, 235, 255, 319, 350, 374, 378...","'[cH][c]([CH2])[cH][c]([cH])[Br]', '[cH][cH][c..."
4,408561088,CCn1c(-c2ccccc2)c(C)s/c1=N\C(=O)c1ccccc1Br,400.024496,5.04852,GMWZBJMZXGQSQF-VZCXRCSSSA-N,GMWZBJMZXGQSQF,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[30, 34, 57, 80, 162, 289, 294, 308, 313, 351,...",'[cH][c]([cH])-[c]([c]([CH3])[s])[n]([c])[CH2]...
...,...,...,...,...,...,...,...,...,...,...
379007,39850378,CCOC(=O)C1=C(C)N=c2s/c(=C\c3ccc(OCC)c(OC)c3)c(...,478.156243,3.20560,NKVUPLXRBGDCKZ-FQWPPLDYSA-N,NKVUPLXRBGDCKZ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[25, 80, 94, 125, 145, 147, 162, 237, 249, 281...","'[c][c]([cH])[O][CH3]', '[CH][c]([cH])[cH][c](..."
379008,606422253,CCCCOc1ccc(-c2nn(-c3ccccc3)cc2CNC(=O)c2ccc(OCC...,501.242770,6.57610,GRVSBYRYFLOHAZ-UHFFFAOYSA-N,GRVSBYRYFLOHAZ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[13, 76, 80, 94, 102, 197, 255, 265, 294, 322,...","'[C][NH][CH2][c]([c])[cH]', '[CH3][CH2][CH2][C..."
379009,19234980,CNc1cc([C@H]2CCCN(Cc3cccc(C)c3)C2)nc(-c2cccnc2)n1,373.226646,4.26832,WGJHTUZEMVLXOF-FQEVSTJZSA-N,WGJHTUZEMVLXOF,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 105, 257, 378, 437, 446, 736, 739, 744, 7...","'[c][cH][c]([n][c])[CH]([CH2])[CH2]', '[CH][CH..."
379010,12371082,COc1ccc(C23CC4CC(CC(C4)C2)C3)cc1C(=O)NC(c1cccc...,451.251129,6.68240,GUGKIBFTNCSUKM-UHFFFAOYSA-N,GUGKIBFTNCSUKM,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 46, 94, 117, 235, 249, 389, 484, 486, 556,...","'[c][CH]([NH])[c]([cH][cH])[cH][cH]', '[c][C](..."


In [5]:
%%time
feature_list_all = []
for x in suppl_csv['Fragments']:
    tup = literal_eval(x)
    feature_list_all.append(tup)
feature_list_all[-1]

CPU times: user 13.8 s, sys: 330 ms, total: 14.1 s
Wall time: 14.1 s


('[CH2][CH2][S](=[O])(=[O])[CH2][NH]',
 '[CH][NH][CH2][S]([CH2])(=[O])=[O]',
 '[CH][CH2][CH2][S]([CH2])(=[O])=[O]',
 '[C][CH]([CH2])[NH][CH2][S]',
 '[CH2][CH]([NH])[C](=[O])[OH]',
 '[CH2][CH2][CH]([NH][CH2])[C](=[O])[OH]',
 '[C][CH]([NH])[CH2][CH2][S]')

In [6]:
feature_list_all[0]

('[cH][cH][c]([CH3])[cH][cH]',
 '[c]-[n]([c])[c](=[O])[c]([c])[C]',
 '[cH][cH][c](-[c]([c])[n])[c]([cH])[CH2]',
 '[c][n]([c])[c]([n][n])[S][CH2]',
 '[cH][cH][c]([cH][cH])-[n]([c])[c]',
 '[cH][cH][cH][c]([c])[CH2]',
 '[c][C]([CH2])([CH2])[CH2][c]([c])[cH]',
 '[c][cH][cH][c]([cH])-[n]',
 '[c][C]([CH2])([CH2])[CH2][CH2][CH2]',
 '[cH][c]([cH])-[n]([c]([c])=[O])[c]([n])[n]',
 '[c]-[c]([c])[n]([c]([n])[S])[c]([n])[n]',
 '[cH][c]([c])-[c]([c]([c])[C])[n]([c])[c]',
 '[c]-[c]([c])[cH][cH][cH]',
 '[c]-[c]([cH])[c]([cH][cH])[CH2][C]',
 '[c]-[n]([c])[c]([n][n])[n]([c])[c]',
 '[CH2][CH2][CH2][CH2][CH2]',
 '[cH][cH][cH][c]([cH])[CH2]',
 '[c][CH2][CH2][S][c]',
 '[c][cH][cH][c]([cH])[CH3]',
 '[c][n][n][c]([n])[S]',
 '[cH][cH][cH][cH][cH]',
 '[cH][c]([cH])[CH2][CH2][S]',
 '[c][n][n][c]([n])[n]',
 '[CH2][CH2][S][c]([n])[n]',
 '[c][cH][cH][cH][cH]',
 '[cH][cH][cH][cH][c]',
 '[C][CH2][CH2][CH2][CH2]',
 '[c][CH2][C]([CH2][CH2])([CH2][CH2])[c]([c])[c]',
 '[cH][cH][c]([cH][cH])[CH2][CH2]',
 '[c]-[c]([n])[c](

In [7]:
pall = CooccurrenceProfile.from_feature_lists(feature_list_all)
pall.df

value
feature1                           feature2                                
[CH2][CH2][CH2][CH2][CH2]          [CH2][CH2][CH2][CH2][CH2]          20003
                                   [CH2][CH2][S][c]([n])[n]             143
                                   [C][CH2][CH2][CH2][CH2]             3587
                                   [cH][cH][cH][cH][cH]                2802
                                   [cH][cH][cH][cH][c]                 4376
...                                                                     ...
[CH][CH2][CH2][S]([CH2])(=[O])=[O] [CH][NH][CH2][S]([CH2])(=[O])=[O]      1
                                   [C][CH]([CH2])[NH][CH2][S]             1
[CH][NH][CH2][S]([CH2])(=[O])=[O]  [C][CH]([CH2])[NH][CH2][S]             1
                                   [C][CH]([NH])[CH2][CH2][S]             1
[C][CH]([CH2])[NH][CH2][S]         [C][CH]([NH])[CH2][CH2][S]             1

[13960003 rows x 1 columns]

In [8]:
pall.df.to_csv('../data/CPF/ZINC/ZINCp_379012.csv')

In [9]:
pall = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/ZINC/ZINCp_379012.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=379012)
pall.df

value
feature1                           feature2                                
[CH2][CH2][CH2][CH2][CH2]          [CH2][CH2][CH2][CH2][CH2]          20003
                                   [CH2][CH2][S][c]([n])[n]             143
                                   [C][CH2][CH2][CH2][CH2]             3587
                                   [cH][cH][cH][cH][cH]                2802
                                   [cH][cH][cH][cH][c]                 4376
...                                                                     ...
[CH][CH2][CH2][S]([CH2])(=[O])=[O] [CH][NH][CH2][S]([CH2])(=[O])=[O]      1
                                   [C][CH]([CH2])[NH][CH2][S]             1
[CH][NH][CH2][S]([CH2])(=[O])=[O]  [C][CH]([CH2])[NH][CH2][S]             1
                                   [C][CH]([NH])[CH2][CH2][S]             1
[C][CH]([CH2])[NH][CH2][S]         [C][CH]([NH])[CH2][CH2][S]             1

[13960003 rows x 1 columns]